In [95]:
!pip install pydub

In [7]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np
import pydub
from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np

C:\Users\Rahul\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [16]:
#define a function to get distance between feature vectors and find neighbors
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist,trainingSet[x]))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x])
#         print(neighbors)
    return neighbors

In [28]:
#function to identify the nearest neighbors
def nearestClass(neighbors,instance):
    classVote = {}
    d1 = neighbors[0][2]
    dk = instance
    k = len(neighbors)
    min1=100
    for x in range(len(neighbors)):
        response = neighbors[x]
        weight = distance(dk,response[2],k)/distance(dk,d1,k)
        if weight<min1:
            min1=weight
            neig = response[0]
#             print(response[0])
#         print(weight)
        if response[0] in classVote:
            classVote[response[0]]+=1 
        else:
            classVote[response[0]]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
#     print(sorter[0][0])
    return neig

In [5]:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

In [8]:
directory = "C:/Users/Rahul/Desktop/IndianS"
# assert os.path.isfile(directory)

f= open("my.dat" ,'wb')
i=0

for folder in os.listdir(directory):
    i+=1
    if i==6 :
        break   
    for file in os.listdir(directory+ "/" + folder):
        #         print(file)  
        (rate,sig) = wav.read(directory+"/"+folder+"/"+file)
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)

f.close()

In [9]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break  
    for x in range(len(dataset)):
        if random.random() <split :      
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])  

trainingSet = []
testSet = []
loadDataset("my.dat" , 0.66, trainingSet, testSet)
# print(testSet)

In [10]:
def distance(instance1, instance2, k):
    distance = 0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

In [33]:
# Make the prediction using KNN(K nearest Neighbors)
leng = len(testSet)
predictions = []
for x in range (leng):
    predictions.append(nearestClass(getNeighbors(trainingSet ,testSet[x] , 2),testSet[x])) 

accuracy1 = getAccuracy(testSet , predictions)
print(accuracy1)

0.8


In [34]:
from collections import defaultdict
results = defaultdict(int)

directory = "C:/Users/Rahul/Desktop/IndianS"

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [35]:
pred = nearestClass(getNeighbors(dataset,feature,5),feature)
print(results[pred])

Sufi
